### **Connecting with Drive**

In [12]:
#Reading the training data Subject
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Importing All needed libraries**

In [13]:
#Importing all needed libraries
import pandas as pd
import numpy as np #Matric math
import tensorflow as tf #ML
from tensorflow.python.framework import ops
from random import randint
from numpy import array
from numpy import argmax
import keras.backend as K
from tensorflow.keras import models
from numpy import array_equal
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import sys
import os
from scipy.io import loadmat
from scipy.io import loadmat

# sys.path.append(os.path.abspath("/Users/henda/anaconda3/Lib/site-packages"))
# from rnn_utils import *
# from public_tests import *
ops.reset_default_graph()

tf.compat.v1.reset_default_graph() #Clearning cache
sess=tf.compat.v1.InteractiveSession()




ERROR:tensorflow:An interactive session is already active. This can cause out-of-memory errors or some other unexpected errors (due to the unpredictable timing of garbage collection) in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s). Please use `tf.Session()` if you intend to productionize.


### **Define Task**

In [14]:
# Split data into support and query sets
def split_task_data(data, sequence_length=15, n_features=96, split_ratio=0.8):
    # X = data[:, 36:58]
    # Z = data[:, 58:82]
    # Z = data[:, 0:60]  # Columns 0 to 59 → 60 columns
    # X = data[:, 60:82] # Columns 60 to 81 → 22 columns
    Z = data[:, 0:96]  # Columns 0 to 59 → 60 columns
    X = data[:, 96:118] # Columns 60 to 81 → 22 columns
    n_sequences = len(Z) - sequence_length + 1

    inputs = np.zeros((n_sequences, sequence_length, n_features))
    outputs = np.zeros((n_sequences, X.shape[1]))

    for j in range(n_sequences):
        inputs[j] = Z[j:j + sequence_length]
        outputs[j] = X[j + sequence_length - 1]

    # Split into support and query sets
    split_index = int(len(inputs) * split_ratio)
    support_x, query_x = inputs[:split_index], inputs[split_index:]
    support_y, query_y = outputs[:split_index], outputs[split_index:]

      # Debugging: Print actual sizes after splitting
    print(f"🚀 Debug: Final Support X = {inputs[:split_index].shape}")
    print(f"🚀 Debug: Final Query X = {inputs[split_index:].shape}")

    return support_x, support_y, query_x, query_y

# Prepare tasks for MAML
def prepare_tasks(subjects, base_path, sequence_length=15, n_features=96):
    tasks = []
    for subject in subjects:
        data_path = f"{base_path}/S{subject}_E1.mat"
        subject_data = loadmat(data_path)["Data"]
        print(f"Shape of subject {subject}: {subject_data.shape}")
        support_x, support_y, query_x, query_y = split_task_data(subject_data, sequence_length, n_features)
        print(f"Shape of support_x for subject {subject}: {support_x.shape}")
        print(f"Shape of support_y for subject {subject}: {support_y.shape}")
        print(f"Shape of query_x for subject {subject}: {query_x.shape}")
        print(f"Shape of query_y for subject {subject}: {query_y.shape}")
        tasks.append((support_x, support_y, query_x, query_y))

    return tasks



### **build model**

In [15]:
from tensorflow.keras import layers, Model, Input

def build_model(input_features=96, timesteps=15, output_dim=22):
    inputs = Input(shape=(timesteps, input_features))

    # CNN Layers
    x = layers.Conv1D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(x)

    x = layers.Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(x)

    x = layers.Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(x)

    # LSTM Layers
    x = layers.Bidirectional(layers.LSTM(500, return_sequences=True))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Bidirectional(layers.LSTM(500))(x)
    x = layers.Dropout(0.2)(x)

    # Fully Connected Layer
    outputs = layers.Dense(output_dim, activation='linear')(x)
    return Model(inputs, outputs)


# # Build LSTM-based model
# def build_model(input_features=24, timesteps=15, output_dim=22):
#     inputs = Input(shape=(timesteps, input_features))
#     x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(inputs)
#     outputs = layers.Dense(output_dim, activation='linear')(x)
#     return Model(inputs, outputs)


### **Training Model framework**

In [16]:
# Inner loop: Task-specific updates
def inner_loop(model, support_x, support_y, loss_fn, inner_lr):
    """
    Perform task-specific updates (inner loop) using the support set.
    Returns updated weights and loss.
    """
    with tf.GradientTape() as tape:
        predictions = model(support_x, training=True)
        loss = loss_fn(support_y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    updated_weights = [w - inner_lr * g for w, g in zip(model.trainable_variables, grads)]
    return updated_weights, loss


# Outer loop: Meta-optimization
def maml_step(model, tasks, loss_fn, meta_optimizer, inner_lr):
    """
    Perform a single meta-update step for MAML using the given tasks.
    """
    meta_grads = [tf.zeros_like(var) for var in model.trainable_variables]

    for support_x, support_y, query_x, query_y in tasks:
        # Perform inner loop to get updated weights
        updated_weights, _ = inner_loop(model, support_x, support_y, loss_fn, inner_lr)

        # Compute query loss using updated weights
        with tf.GradientTape() as tape:
            # Temporarily set the model's weights to the updated weights
            query_predictions = model(query_x, training=True)
            query_loss = loss_fn(query_y, query_predictions)

        # Compute gradients with respect to the original weights
        task_grads = tape.gradient(query_loss, model.trainable_variables)

        # Accumulate gradients across all tasks
        meta_grads = [meta_grad + task_grad for meta_grad, task_grad in zip(meta_grads, task_grads)]

    # Apply meta-gradient updates
    meta_grads = [grad / len(tasks) for grad in meta_grads]  # Average gradients
    meta_optimizer.apply_gradients(zip(meta_grads, model.trainable_variables))


### **Train Model**

In [17]:
# MAML Training Loop
def maml_train(model, tasks, epochs, inner_lr, meta_lr):
    loss_fn = tf.keras.losses.MeanSquaredError()
    meta_optimizer = tf.keras.optimizers.Adam(meta_lr)

    for epoch in range(epochs):
        maml_step(model, tasks, loss_fn, meta_optimizer, inner_lr)
        print(f"Epoch {epoch + 1}/{epochs} completed")

###############   MAML +Ensemble learning ###########

def maml_train_ensemble(num_models, tasks, epochs, inner_lr, meta_lr):
    """
    Train multiple MAML models (ensemble members) on the given tasks.

    Parameters:
    - num_models: Number of models in the ensemble.
    - tasks: List of tasks for meta-training.
    - epochs: Number of meta-training epochs.
    - inner_lr: Inner-loop learning rate.
    - meta_lr: Meta-learning rate for the outer loop.

    Returns:
    - ensemble: List of trained models.
    """
    ensemble = []

    for i in range(num_models):
        print(f"Training model {i + 1}/{num_models}...")
        model = build_model()  # Use the CNN-LSTM model
        maml_train(model, tasks, epochs, inner_lr, meta_lr)  # Meta-train the model
        ensemble.append(model)

    return ensemble





### **Testing**

In [18]:
# # Fine-Tune Each Model
# def fine_tune_ensemble(ensemble, support_x, support_y, loss_fn, inner_lr, fine_tune_steps=1):
#     fine_tuned_weights = []
#     for i, model in enumerate(ensemble):
#         print(f"Fine-tuning model {i + 1}/{len(ensemble)}...")
#         updated_weights = model.trainable_variables
#         for _ in range(fine_tune_steps):
#             updated_weights, _ = inner_loop(model, support_x, support_y, loss_fn, inner_lr)
#         fine_tuned_weights.append(updated_weights)
#     return fine_tuned_weights

def fine_tune_ensemble(ensemble, support_x, support_y, loss_fn, inner_lr, fine_tune_steps=1, batch_size=512):
    """
    Fine-tune the ensemble models using a smaller batch size to avoid GPU memory errors.
    """
    fine_tuned_weights = []

    for i, model in enumerate(ensemble):
        print(f"🚀 Fine-tuning model {i + 1}/{len(ensemble)}...")

        updated_weights = model.trainable_variables
        dataset = tf.data.Dataset.from_tensor_slices((support_x, support_y)).batch(batch_size)  # ✅ Use batching

        for _ in range(fine_tune_steps):
            for batch_x, batch_y in dataset:  # ✅ Process in small batches
                updated_weights, _ = inner_loop(model, batch_x, batch_y, loss_fn, inner_lr)

        fine_tuned_weights.append(updated_weights)

    return fine_tuned_weights


    # Combine Predictions from Ensemble
def ensemble_predict(ensemble, fine_tuned_weights, query_x):
    all_predictions = []
    for i, model in enumerate(ensemble):
        print(f"Predicting with model {i + 1}/{len(ensemble)}...")
        predictions = model(query_x, training=True)
        all_predictions.append(predictions)
    ensemble_predictions = np.mean(all_predictions, axis=0)
    return ensemble_predictions

def ensemble_predict_weighted(ensemble, fine_tuned_weights, query_x, weights):
    """
    Combine predictions from the ensemble using weighted averaging.

    Parameters:
    - ensemble: List of models.
    - fine_tuned_weights: Fine-tuned weights for each model.
    - query_x: Input features for the query set.
    - weights: List of weights for each model.

    Returns:
    - ensemble_predictions: Weighted average predictions.
    """
    all_predictions = []
    for i, model in enumerate(ensemble):
        print(f"Predicting with model {i + 1}/{len(ensemble)}...")
        predictions = model(query_x, training=True)
        all_predictions.append(predictions)

    # Convert to numpy array for weighted averaging
    all_predictions = np.array(all_predictions)  # Shape: (num_models, batch_size, output_dim)
    weights = np.array(weights).reshape(-1, 1, 1)  # Reshape to broadcast

    # Weighted average
    ensemble_predictions = np.sum(all_predictions * weights, axis=0) / np.sum(weights)
    return ensemble_predictions

def ensemble_predict_closest(ensemble, fine_tuned_weights, query_x, query_y):
    """
    Combine predictions from the ensemble by selecting the model closest to actual values.

    Parameters:
    - ensemble: List of models.
    - fine_tuned_weights: Fine-tuned weights for each model.
    - query_x: Input features for the query set.
    - query_y: Ground truth values for the query set.

    Returns:
    - ensemble_predictions: Predictions closest to the actual values.
    """
    all_predictions = []
    errors = []

    for i, model in enumerate(ensemble):
        print(f"Predicting with model {i + 1}/{len(ensemble)}...")
        predictions = model(query_x, training=True)
        all_predictions.append(predictions)

        # Calculate mean squared error for this model
        error = np.mean((predictions - query_y) ** 2, axis=0)  # Error per DOF
        errors.append(np.sum(error))  # Total error for this model

    # Find the model with the least error
    best_model_index = np.argmin(errors)
    print(f"Best model is model {best_model_index + 1} with error {errors[best_model_index]:.4f}")

    # Use the best model's predictions
    return all_predictions[best_model_index]

def ensemble_predict_error_weighted(ensemble, fine_tuned_weights, query_x, query_y):
    """
    Combine predictions from the ensemble using error-weighted averaging.

    Parameters:
    - ensemble: List of models.
    - fine_tuned_weights: Fine-tuned weights for each model.
    - query_x: Input features for the query set.
    - query_y: Ground truth values for the query set.

    Returns:
    - ensemble_predictions: Error-weighted average predictions.
    """
    all_predictions = []
    errors = []

    for i, model in enumerate(ensemble):
        print(f"Predicting with model {i + 1}/{len(ensemble)}...")
        predictions = model(query_x, training=True)
        all_predictions.append(predictions)

        # Calculate mean squared error for this model
        error = np.mean((predictions - query_y) ** 2, axis=0)  # Error per DOF
        errors.append(np.sum(error))  # Total error for this model

    # Compute weights inversely proportional to errors
    weights = 1 / (np.array(errors) + 1e-8)  # Avoid division by zero
    weights = weights / np.sum(weights)  # Normalize weights

    # Convert predictions to numpy for weighted averaging
    all_predictions = np.array(all_predictions)  # Shape: (num_models, batch_size, output_dim)

    # Weighted average
    ensemble_predictions = np.sum(all_predictions * weights[:, None, None], axis=0)
    return ensemble_predictions

def ensemble_predict_weighted_performance(ensemble, fine_tuned_weights, query_x, query_y):
    all_predictions = []
    errors = []

    for i, model in enumerate(ensemble):
        print(f"Predicting with model {i + 1}/{len(ensemble)}...")
        predictions = model(query_x, training=True)
        if predictions is None:
            print(f"Error: Model {i + 1} returned None predictions.")
            return None
        all_predictions.append(predictions)

        # Calculate mean squared error for this model
        error = np.mean((predictions - query_y) ** 2, axis=0)  # Error per DOF
        errors.append(np.sum(error))  # Total error for this model

    # Compute weights inversely proportional to errors
    weights = 1 / (np.array(errors) + 1e-8)  # Avoid division by zero
    weights = weights / np.sum(weights)  # Normalize weights

    # Convert predictions to numpy array for weighted averaging
    all_predictions = np.array(all_predictions)  # Shape: (num_models, batch_size, output_dim)
    if all_predictions.size == 0:
        print("Error: No predictions available for ensemble. Check models or input data.")
        return None

    # Weighted average
    ensemble_predictions = np.sum(all_predictions * weights[:, None, None], axis=0)
    print(f"Model Weights (based on performance): {weights}")
    return ensemble_predictions


# # Fine-tune on a new subject
# def fine_tune(model, support_x, support_y, loss_fn, inner_lr, fine_tune_steps=1):
#     """
#     Fine-tune the model on the support set for a new task.
#     Returns the updated weights.
#     """
#     updated_weights = model.trainable_variables
#     for _ in range(fine_tune_steps):
#         updated_weights, _ = inner_loop(model, support_x, support_y, loss_fn, inner_lr)
#     return updated_weights


# # Evaluate on a new subject
# def evaluate_model(model, updated_weights, query_x, query_y):
#     """
#     Evaluate the model on the query set using the updated weights.
#     """
#     # Temporarily set the model's weights to the updated weights
#     query_predictions = model(query_x, training=True)
#     correlations = [np.corrcoef(query_y[:, i], query_predictions[:, i])[0, 1] for i in range(query_y.shape[1])]
#     avg_correlation = np.mean(correlations)
#     print(f"Average Pearson correlation: {avg_correlation}")
#     return avg_correlation



### **Plot the result**

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


def normalize_predictions(predictions):
    mean = np.mean(predictions, axis=0)
    std = np.std(predictions, axis=0) + 1e-8  # Avoid division by zero
    return (predictions - mean) / std
def scale_predictions(predictions):
    min_val = np.min(predictions, axis=0)
    max_val = np.max(predictions, axis=0)
    return (predictions - min_val) / (max_val - min_val)
def smooth_predictions(predictions, window_size=5):
    smoothed = np.zeros_like(predictions)
    for i in range(predictions.shape[1]):  # Loop through each DOF
        smoothed[:, i] = np.convolve(predictions[:, i], np.ones(window_size)/window_size, mode='same')
        smoothed[:, i] = np.convolve(predictions[:, i], np.ones(window_size)/window_size, mode='same')
    return smoothed

def evaluate_and_plot(model, updated_weights, query_x, query_y, apply_smoothing=True, apply_normalization=True,apply_Scale=True):
    """
    Evaluate the model for each DOF, calculate CC, and plot actual vs. normalized or smoothed predictions.

    Parameters:
    - model: The trained model.
    - updated_weights: Fine-tuned weights for the task.
    - query_x: Input features for the query set.
    - query_y: Ground truth outputs for the query set.
    - apply_smoothing: Whether to smooth predictions.
    - apply_normalization: Whether to normalize predictions.

    Returns:
    - correlations: List of CC values for each DOF.
    - avg_correlation: Average CC across all DOFs.
    """
    # Predict using the updated weights
    query_predictions = model(query_x, training=True)

    # Normalize or smooth predictions

    if apply_smoothing:
        query_predictions = smooth_predictions(query_predictions)
    if apply_normalization:
        query_predictions = normalize_predictions(query_predictions)
    if apply_Scale:
        query_predictions = scale_predictions(query_predictions)

    # Initialize lists for storing CC values
    correlations = []

    # Loop through each DOF (dimension)
    for i in range(query_y.shape[1]):
        # Calculate Pearson Correlation Coefficient
        cc_value, _ = pearsonr(query_y[:, i], query_predictions[:, i])
        correlations.append(cc_value)

        # Plot Actual vs Predicted for the current DOF
        plt.figure(figsize=(10, 6))
        plt.plot(query_y[:, i], label='Actual', color='blue')
        plt.plot(query_predictions[:, i], label='Predicted', color='red')

        # Add title and labels
        plt.title(f'Degree of Freedom {i + 1}: CC = {cc_value:.2f}')
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()
        plt.show()

    # Compute the average CC across all DOFs
    avg_correlation = np.mean(correlations)
    print(f"Average Pearson Correlation Coefficient: {avg_correlation:.2f}")

    return correlations, avg_correlation

def evaluate_ensemble(ensemble_predictions, query_y, apply_smoothing=True, apply_normalization=True, apply_scale=True):
    """
    Evaluate ensemble predictions and plot results.

    Parameters:
    - ensemble_predictions: Predictions from the ensemble.
    - query_y: Ground truth for the query set.
    - apply_smoothing: Whether to smooth predictions.
    - apply_normalization: Whether to normalize predictions.
    - apply_scale: Whether to scale predictions.

    Returns:
    - correlations: List of CC values for each DOF.
    - avg_correlation: Average CC across all DOFs.
    """
    # Process predictions
    if apply_smoothing:
        ensemble_predictions = smooth_predictions(ensemble_predictions)
    if apply_normalization:
        ensemble_predictions = normalize_predictions(ensemble_predictions)
    if apply_scale:
        ensemble_predictions = scale_predictions(ensemble_predictions)

    # Initialize lists for storing CC values
    correlations = []

    # Loop through each DOF (dimension)
    for i in range(query_y.shape[1]):
        # Calculate Pearson Correlation Coefficient
        cc_value, _ = pearsonr(query_y[:, i], ensemble_predictions[:, i])
        correlations.append(cc_value)

        # Plot Actual vs Predicted for the current DOF
        plt.figure(figsize=(10, 6))
        plt.plot(query_y[:, i], label='Actual', color='blue')
        plt.plot(ensemble_predictions[:, i], label='Predicted', color='red')

        # Add title and labels
        plt.title(f'Degree of Freedom {i + 1}: CC = {cc_value:.2f}')
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()
        plt.show()

    # Compute the average CC across all DOFs
    avg_correlation = np.mean(correlations)
    print(f"Average Pearson Correlation Coefficient: {avg_correlation:.2f}")

    return correlations, avg_correlation



## **Methods for saving the data**

In [20]:
# Save data in Excel file
def savedata(i, correlations):
    """
    Save the correlations data in an Excel file at the specified row.

    Parameters:
    - i: Integer representing the subject number (e.g., 5 for row name 'S5').
    - correlations: List of correlation values to save in the row.
    """
    import openpyxl
    from google.colab import drive

    # Mount Google Drive
    drive.mount('/content/drive')

    # Path to the workbook
    workbook_path = '/content/drive/My Drive/Colab Notebooks/dataset/Ninapro MAML+weighted ensemble_CNN-LSTM + extra features Subject Independant.xlsx'

    # Load the existing workbook
    wb = openpyxl.load_workbook(workbook_path)

    # Select the active sheet
    sheet = wb.active

    # Dynamically set the row name based on 'i'
    row_name = f'S{i}'  # Example: 'S5' if i=5

    # Find the row with the specified row name
    target_row = None
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row):
        if row[0].value == row_name:
            target_row = row[0].row
            break

    if target_row is None:
        print(f"Error: Row with name '{row_name}' not found in the sheet.")
        return

    # Write the data to the found row, starting from the second column
    for col, value in enumerate(correlations, start=2):  # Start at column 2 to skip the first column for the label
        sheet.cell(row=target_row, column=col, value=value)

    # Save the workbook
    wb.save(workbook_path)
    print(f"Data successfully saved for row '{row_name}'.")



In [21]:
import random
def process_subject(test_subject, ensemble, base_path,train_subjects,fine_tune_subjects=5):
    """
    Process a single test subject:
    - Fine-tune the ensemble on the support set of the test subject.
    - Combine predictions from the ensemble.
    - Evaluate and visualize results.
    - Return correlations and average correlation.

    Parameters:
    - test_subject: Subject to be tested (integer).
    - ensemble: List of models in the ensemble.
    - base_path: Path to the dataset.

    Returns:
    - correlations: List of correlation coefficients for each DOF.
    - avg_correlation: Average correlation across all DOFs.
    """
    print(f"Processing Test Subject: S{test_subject}")

    # Load test subject data
    test_data = loadmat(f"{base_path}/S{test_subject}_E1.mat")["Data"]

    # Debugging: Check original dataset shape
    print(f"🚀 Debug: Test Subject {test_subject} Original Data Shape = {test_data.shape}")

    # DO NOT SPLIT TEST SUBJECT INTO SUPPORT/QUERY!
    Z = test_data[:, :96]  # Input features
    X = test_data[:, 96:118]  # DOFs (outputs)

    # Reshape for model input
    sequence_length = 15
    n_sequences = len(Z) - sequence_length + 1

    query_x = np.zeros((n_sequences, sequence_length, 96))
    query_y = np.zeros((n_sequences, X.shape[1]))

    for j in range(n_sequences):
        query_x[j] = Z[j:j + sequence_length]
        query_y[j] = X[j + sequence_length - 1]

    # Debugging: Verify dataset is NOT being shrunk
    print(f"🚀 Debug: Using Full Test Data - Query X = {query_x.shape}, Query Y = {query_y.shape}")

      # Choose a subset of subjects for fine-tuning
    fine_tune_subjects_list = random.sample([s for s in train_subjects if s != test_subject], fine_tune_subjects)
    print(f"🚀 Fine-tuning with {fine_tune_subjects} subjects: {fine_tune_subjects_list}")

    support_x_list = []
    support_y_list = []

    for subject in fine_tune_subjects_list:
        train_data = loadmat(f"{base_path}/S{subject}_E1.mat")["Data"]
        support_x, support_y, _, _ = split_task_data(train_data)  # Train subjects only
        support_x_list.append(support_x)
        support_y_list.append(support_y)

    support_x = np.concatenate(support_x_list, axis=0)
    support_y = np.concatenate(support_y_list, axis=0)

    print(f"🚀 Debug: Training Support Dataset - Support X = {support_x.shape}, Support Y = {support_y.shape}")

    # Fine-tune ensemble using reduced support dataset
    fine_tuned_weights = fine_tune_ensemble(
        ensemble,
        support_x,
        support_y,
        loss_fn=tf.keras.losses.MeanSquaredError(),
        inner_lr=0.01,
        fine_tune_steps=1
    )

    # Combine predictions using performance-weighted averaging
    ensemble_predictions = ensemble_predict_weighted_performance(
        ensemble, fine_tuned_weights, query_x, query_y
    )

    # Evaluate and visualize results
    print(f"Debug: query_predictions shape: {ensemble_predictions.shape}, query_y shape: {query_y.shape}")

    correlations, avg_correlation = evaluate_ensemble(
        ensemble_predictions,
        query_y,
        apply_smoothing=True,  # Enable smoothing
        apply_normalization=False,  # Disable normalization if not needed
        apply_scale=True            # Enable scaling
    )

    # Print results
    print(f"Correlation Coefficient for each DOF (S{test_subject}): {correlations}")
    print(f"Average Correlation Coefficient (S{test_subject}): {avg_correlation:.2f}")
    print("-" * 50)

    return correlations, avg_correlation


### **Main method**

In [22]:
# if __name__ == "__main__":
#     base_path = '/content/drive/My Drive/Colab Notebooks/dataset wstimi'
#     all_subjects = list(range(1, 41))  # Subjects 1 to 40

#     for test_subject in all_subjects:
#         print(f"Starting Processing for Test Subject: S{test_subject}")

#         # Exclude the test subject from training subjects
#         train_subjects = [s for s in all_subjects if s != test_subject]

#         # Prepare tasks for training
#         tasks = prepare_tasks(train_subjects, base_path)

#         # Train the ensemble of models
#         num_models = 5  # Number of models in the ensemble
#         ensemble = maml_train_ensemble(num_models, tasks, epochs=50, inner_lr=0.01, meta_lr=0.001)

#         # Process the current test subject
#         correlations, avg_correlation = process_subject(test_subject, ensemble, base_path)

#         # Save the results
#         savedata(test_subject, correlations)

import os

if __name__ == "__main__":
    base_path = '/content/drive/My Drive/Colab Notebooks/dataset wstimi'
    all_subjects = list(range(1, 41))  # Subjects 1 to 40

    # Start resuming from test subject 10
    start_subject = 1

    for test_subject in all_subjects:
        # Skip subjects that were already processed
        if test_subject < start_subject:
            continue  # Skip to the starting subject

        print(f"Starting Processing for Test Subject: S{test_subject}")

        # Ensure all other subjects (1–40) except the current one are in the training set
        train_subjects = [s for s in all_subjects if s != test_subject]

        # Prepare tasks for training
        tasks = prepare_tasks(train_subjects, base_path)

        # Train the ensemble of models
        num_models = 1  # Number of models in the ensemble
        ensemble = maml_train_ensemble(num_models, tasks, epochs=70, inner_lr=0.01, meta_lr=0.001)

        # Process the current test subject
        correlations, avg_correlation = process_subject(test_subject, ensemble, base_path,train_subjects,5)

        # Save the results
        # savedata(test_subject, correlations)



Output hidden; open in https://colab.research.google.com to view.